In [27]:
import requests

url = "http://localhost:8081/personalized_recommendation"
payload = {
    "courses": ["SF222"],
    "skills": ["Python", "Data Analysis", "Angular"]
}

try:
    response = requests.post(url, data=payload)
    response.raise_for_status()  # Raise an exception for bad status codes
    recommendations = response.json()
    print(recommendations)
except requests.exceptions.RequestException as e:
    print(f"Error sending request: {e}")
except ValueError:
    print("Error decoding JSON response.")

{'job_fulfillment': {'Backend (Bank) #1': {'Windows': True, 'Unix': True, 'Linux': True, 'PL/SQL': False, 'Oracle': False, 'DB2': False, 'UML': True, 'Websphere': False, 'JBOSS': False, 'JSP': False, 'Servlet': False, 'J2EE': False, 'J2SE': False, 'Struts': False, 'Spring': False, 'Hibernate': False, 'JBuilder': False, 'Netbean': False, 'Eclipse': False, 'JQuery': True, 'AJAX': True, 'Javascript': True, 'Banking Business': False, 'IT Industry': False, 'Java Developer': False, 'Object-Oriented Programming': False, 'Object-Oriented Design': False, 'Software Development Life Cycle': False, 'System Analysis': True, 'Java': True}, 'Backend (Bank) #2': {'CSS': True, 'HTML5': True, 'Android': True, 'iOS': True, 'Web Development': True, 'Mobile Development': True, 'Git': True, 'CI/CD': True, 'MongoDB': True, 'MySQL': True, 'NoSQL Database': True, 'SQL': True, 'Spring Cloud': False, 'Spring Boot': False, 'Spring Framework': True, 'JSON': True, 'XML': True, 'SOAP': True, 'REST': True, 'Web Servi

In [37]:
import pandas as pd

df = pd.DataFrame(recommendations['sankey_df'])
grp = df.groupby('job')

top5jobs = [v[0] for v in recommendations['ranked_jobs'][:5]]

S = set()
for job in top5jobs:
    g = grp.get_group(job)
    s = set([tuple(v) for v in g.data.values])
    S = S.union(s)
len(S)

257

In [29]:
recommendations['ranked_jobs']

[['Backend (Travel) #2', 0.9444444444444444],
 ['Backend (Bank) #2', 0.8484848484848485],
 ['Fullstack (Insurance) #1', 0.8333333333333334],
 ['Fullstack (Bank) #2', 0.625],
 ['Fullstack (EdTech) #1', 0.6176470588235294],
 ['Backend (Travel) #1', 0.6052631578947368],
 ['Frontend (Consult) #1', 0.575],
 ['Fullstack (FinTech) #2', 0.5263157894736842],
 ['Backend (FinTech) #2', 0.5],
 ['Fullstack (FinTech) #1', 0.5],
 ['Fullstack (MedTech) #1', 0.45652173913043476],
 ['Fullstack (Bank) #1', 0.4411764705882353],
 ['Backend (Bank) #1', 0.3],
 ['PM (MedTech) #1', 0.21428571428571427],
 ['QA (FinTech) #1', 0.12],
 ['Infra (Travel) #1', 0.09803921568627451],
 ['QA (MedTech) #1', 0.047619047619047616]]

In [16]:
recommendations['ranked_jobs']

[['Backend (Travel) #2', 1.0],
 ['Fullstack (EdTech) #1', 0.8823529411764706],
 ['Backend (Bank) #2', 0.8787878787878788],
 ['Fullstack (MedTech) #1', 0.8695652173913043],
 ['Backend (Travel) #1', 0.868421052631579],
 ['Fullstack (Insurance) #1', 0.8333333333333334],
 ['PM (MedTech) #1', 0.7857142857142857],
 ['Fullstack (FinTech) #1', 0.7666666666666667],
 ['Fullstack (Bank) #2', 0.75],
 ['Frontend (Consult) #1', 0.7],
 ['Backend (FinTech) #2', 0.65],
 ['Fullstack (Bank) #1', 0.6176470588235294],
 ['Fullstack (FinTech) #2', 0.5789473684210527],
 ['Backend (Bank) #1', 0.4666666666666667],
 ['QA (FinTech) #1', 0.44],
 ['Infra (Travel) #1', 0.23529411764705882],
 ['QA (MedTech) #1', 0.1111111111111111]]

In [40]:
def find_and_break_cycles(edges):
    adj = {}
    for u, v in edges:
        adj.setdefault(u, []).append(v)

    visited = set()
    recursion_stack = set()
    cycle_edges = set()

    def detect_cycle(node, path):
        visited.add(node)
        recursion_stack.add(node)
        path.append(node)

        for neighbor in adj.get(node, []):
            if neighbor not in visited:
                if detect_cycle(neighbor, path):
                    return True
            elif neighbor in recursion_stack:
                # Cycle detected! The edge (node, neighbor) is part of a cycle.
                # We'll mark it for removal.
                cycle_edges.add((node, neighbor))
                return True
        recursion_stack.remove(node)
        path.pop()
        return False

    all_nodes = set([u for u, v in edges] + [v for u, v in edges])
    for node in all_nodes:
        if node not in visited:
            detect_cycle(node, [])

    return cycle_edges

sankey_data = [('a', 'b'), ('b', 'c'), ('d', 'e'), ('e', 'a')]
edges_to_remove = find_and_break_cycles(sankey_data)
print(f"Edges to remove to break cycles: {edges_to_remove}")

# To get the graph without cycles, you can filter the original list:
acyclic_data = [edge for edge in sankey_data if edge not in edges_to_remove]
print(f"Graph data without cycles: {acyclic_data}")

Edges to remove to break cycles: set()
Graph data without cycles: [('a', 'b'), ('b', 'c'), ('d', 'e'), ('e', 'a')]
